Execute a célula de baixo se ainda não instalou o `Watson Developer Cloud Python SDK`. Essa é a biblioteca que vai te permitir conectar, via Python, aos serviços da IBM Cloud.

In [ ]:
!pip install --upgrade ibm-watson

Após a instalação, você deve importar os seguintes serviços suficientes para o desafio:

In [ ]:
# Lidar com arquivos JSON
import json

# Aqui importamos as classes que cuidam dos serviços do Natural Language Understanding e do Speech-to-Text
from ibm_watson import NaturalLanguageUnderstandingV1, SpeechToTextV1

# Puxamos, para o natural language understanding, as classes de Features e EntitiesOptions que serão úteis para obter as entidades e os sentimentos associdados
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions

# Só é possível se conectar aos seus serviços se você se autenticar, e a classe que cuidará disso é o IAMAuthenticator
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## Serviço NLU

Você precisará de 3 coisas: A key e a URL do seu serviço de `Natural Language Understanding` e o model_id do seu Knowledge Studio treinado.

In [ ]:
nlu_apikey = "sua-key-aqui"
    
nlu_service_url = "sua-url-aqui"
    
nlu_entity_model = "seu-model-id-aqui"

In [2]:
nlu_apikey = "NyDMAj43hZvu1T-vWbz-evJgiYhZZZwR3_bLO1-y0P7f"
nlu_service_url = "https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/7e56160b-4784-41fb-8d56-193a3b3c7fec"  
nlu_entity_model = "4f0c3f0d-d6ec-4723-a3ae-1868a6435c69"

# Speech-to-Text
stt_apikey = "HfKqTA4ixXjPkNfKNSFcK6P-ATgmGuhXIqOTOxhO0RYM"
stt_service_url = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/c787c51d-d355-48a4-898e-96e61da2d7ad"
stt_entity_model = 'pt-BR_BroadbandModel'


Agora instanciamos os serviços com as suas credenciais.

In [3]:
# Cria-se um autenticador
nlu_authenticator = IAMAuthenticator(apikey=nlu_apikey)

# Criamos o serviço passando esse autenticador
nlu_service = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    authenticator=nlu_authenticator)

# Setamos a URL de acesso do nosso serviço
nlu_service.set_service_url(nlu_service_url)

## Serviço STT

Aqui você só vai precisar da key e da URL do seu serviço do `Speech-to-Text`

In [ ]:
stt_apikey = "sua-key-aqui"

stt_service_url = "sua-url-aqui"

E agora instanciamos o serviço com as suas credenciais.

In [4]:
stt_authenticator = IAMAuthenticator(apikey=stt_apikey)

stt_service = SpeechToTextV1(authenticator=stt_authenticator)

stt_service.set_service_url(stt_service_url)

## Resposta NLU

Para usar o serviço agora, basta mandar uma variável contendo seu texto para o serviço utilizando o método `.analyze()`. É no próprio método que, na parte de `Features`, você define o modelo que quer usar para extrair as entidades e o sentimento. Lembre que a variável deve estar do formato `str`, ou seja, deve ser uma string. 

Quando rodar a célula abaixo, você verá o resultado do `nlu_response` logo em seguida. Deve demorar alguns segundos para fazer a requisição.

In [ ]:
# Defina o que você quer enviar para o seu modelo analisar. O importante é que você envie uma string. Essa variável text pode ser o que você quiser
text = "O novo Fiat Toro apresenta um motor incrível! Sem dúvida possui um dos melhores arranques que eu conheço. O design exterior do veículo também é de chamar muita atenção!"

# O método analyze cuida de tudo
nlu_response = nlu_service.analyze(
    text=text,
    features=Features(entities=EntitiesOptions(model=nlu_entity_model, sentiment=True)),
    language='pt'
).get_result()

print(json.dumps(nlu_response, indent=2, ensure_ascii=False))

Uma vez feito isso, sua variável `nlu_result` é agora um `dict` (dicionário) com a resposta do serviço ao texto que você forneceu. Você pode agora manipular como quiser essa resposta em seu código para completar o desafio.

## Resposta STT

Aqui é a mesma ideia, mas no caso como enviamos um arquivo temos que abrí-lo primeiro (quando você estiver montando sua api você vai receber como requisição e não como leitura do disco, isso é apenas para exemplo). Você deve enviar um arquivo de tipo `bytes` para o serviço. O `open` do python cuida dessa parte para esse caso. Não carregue um arquivo que não seja formato `.flac`, pois não irá funcionar.

In [6]:
caminho_para_audio = "./doc/source/dataset/audio_sample.flac"
#caminho_para_audio = "coloque o PATH da amostra de áudio no formato flac aqui"

Como o serviço de Speech-to-Text lida com linguagem, você deve indicar qual modelo linguístico você quer usar. Queremos utilizar um que identifique o português do Brasil. Eu já o separei abaixo para você.

In [ ]:
stt_model = 'pt-BR_BroadbandModel'

Com isso pronto, carregamos nosso arquivo `.flac` e o enviamos ao serviço. Isso é feito agora pelo método `.recognize()`. Feito isso você verá o resultado ser printado logo abaixo.

In [7]:
with open(caminho_para_audio, 'rb') as audio_file:
    stt_results = stt_service.recognize(
            audio=audio_file,
            content_type='audio/flac',
            #model=stt_model,
            timestamps=False,
            word_confidence=False).get_result()
    
print(json.dumps(stt_results,indent=2, ensure_ascii=False))

TypeError: 'indent' is an invalid keyword argument for this function

Uma vez feito isso, sua variável `stt_results` é agora um `dict` (dicionário) com a resposta do serviço ao arquivo de áudio que você forneceu. Você pode agora manipular como quiser essa resposta em seu código para completar o desafio.

## Boa Sorte!!